# LSTM + word2vec

#### Preprosessing

nltk preprocessing, keras default tokenizer (remove stopwords, numbers, punctuation)

#### Model

LSTM base - one layer (100 - hidden units, without dropouts) 
* 1) word2vec train own texts
* 2) pretrained word2vec

---

In [69]:
MODEL_NAME = "imdb_lstm_word2vec"
TRAIN_DATASET = "train_imdb.csv"
TEST_DATASET = "test_imdb.csv"

TOKENIZER_NAME = "lstm_imdb_tokenizer"

---

In [70]:
import nltk
import numpy
import pickle
numpy.random.seed(42)

from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

Swith on full text mode)

In [71]:
pd.options.display.max_colwidth = -1

## Load data

In [72]:
train_data = pd.read_csv(TRAIN_DATASET, sep="|")

In [73]:
test_data = pd.read_csv(TEST_DATASET, sep="|")

---

## Preprocessing

In [74]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 50

---

Remove stopwords, numbers, punctuation

In [75]:
stops = set(stopwords.words('english'))

def preprocess(text):
    return ' '.join(list(filter(lambda x: x.isalnum(), nltk.word_tokenize(str(text.lower())))))

In [76]:
train_data['prep_text'] = train_data['text'].map(preprocess)
test_data['prep_text'] = test_data['text'].map(preprocess)

In [77]:
train_data.head()

,Unnamed: 0,label,text,prep_text
0,48903,1,"Although this movie has some weaknesses, it is worth seeing. I chose it because of the cast, and applaud Bonham Carter and Branagh for choosing roles different from those they have taken in the past. Both portray very troubled people, complete with warts, but make them likeable because of their humanity. The story is touching, but it is the performances that soar. Bonham Carter's ""Jane"" is a remarkable achievement, whose quest for romance opened my eyes to aspects of being disabled that I had not thought of before, but was interesting as well for other reasons. I felt the movie ended too abruptly, but better that than a drawn out emotionally manipulative ending (see ""Stepmom."") The very real English setting added to my enjoyment - it was England in the 90's, both urban and rural, without being depressing.",although this movie has some weaknesses it is worth seeing i chose it because of the cast and applaud bonham carter and branagh for choosing roles different from those they have taken in the past both portray very troubled people complete with warts but make them likeable because of their humanity the story is touching but it is the performances that soar bonham carter jane is a remarkable achievement whose quest for romance opened my eyes to aspects of being disabled that i had not thought of before but was interesting as well for other reasons i felt the movie ended too abruptly but better that than a drawn out emotionally manipulative ending see stepmom the very real english setting added to my enjoyment it was england in the 90 both urban and rural without being depressing
1,10174,0,"I hate to comment on something I didn't finish, but if I spare one person what I sat through for almost an hour before turning it off in disgust, it will be worth it. <br /><br />I decided to watch this with an open mind, knowing it was on the bottom 100.<br /><br />Bad idea. I usually love crude humor, or can at least tolerate it. I love so-called ""black"" comedies. I'm not easily offended, either. <br /><br />It started off okay and quickly went downhill. I laughed a few times (for example, when the main character got stuck in the airplane toilet), but that was it and didn't compensate for the strong disgust I felt.<br /><br />I didn't laugh when the dog got sucked into a jet engine. I usually can't see the humor in animals dying (except in Animal House). I didn't laugh at much else of the nastiness, either. I turned it off after an incident involving a blind man and a baked potato that I don't care to repeat the details of, only that the wave of nausea still hasn't passed over me. <br /><br />Simply put, it was smut-filled and simply not funny with barely any plot. This is one of the times when if you don't have something nice to say, you should get the word out.<br /><br />Don't say I didn't warn you.",i hate to comment on something i did finish but if i spare one person what i sat through for almost an hour before turning it off in disgust it will be worth it br br i decided to watch this with an open mind knowing it was on the bottom br br bad idea i usually love crude humor or can at least tolerate it i love black comedies i not easily offended either br br it started off okay and quickly went downhill i laughed a few times for example when the main character got stuck in the airplane toilet but that was it and did compensate for the strong disgust i br br i did laugh when the dog got sucked into a jet engine i usually ca see the humor in animals dying except in animal house i did laugh at much else of the nastiness either i turned it off after an incident involving a blind man and a baked potato that i do care to repeat the details of only that the wave of nausea still has passed over me br br simply put it was and simply not funny with barely any plot this is one of the times when if you do have something nice to say you should get the word br br do say i did warn you
2,34019,0,"The m

### Padding data

Keras [Embedding layer](https://keras.io/layers/embeddings/) turn positive integers (indexes) into dense vectors of fixed size. 

* 1) Firstly convert words to indexes
* 2) Then we padding data

['юристы есть', 'мне нужны юристы'] -> [[0, 4, 10], [2, 3, 4]] -> [[0.25, 0.1], [0.6, -0.2]]

#### Train own word2vec

In [10]:
%%time
import gensim
from gensim.models import Word2Vec

word2vec_sequences = []
for raw_sentence in pd.concat([train_data['prep_text'], test_data['prep_text']]):
    # If a sentence is empty, skip it
    if len(raw_sentence) > 0:
        # Otherwise, call review_to_wordlist to get a list of words
        word2vec_sequences.append(raw_sentence.split())

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 10    # Minimum word count                        
num_workers = 8       # Number of threads to run in parallel
context = 5          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
training_algorithm = 1 # 0-CBOW, 1-SkipGram 

word2vec_own = Word2Vec(word2vec_sequences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, \
            sg=training_algorithm)

CPU times: user 1min 42s, sys: 804 ms, total: 1min 43s
Wall time: 15.5 s


In [11]:
len(word2vec_own.index2word)

12272

In [12]:
word2vec_own.most_similar('man')

[('woman', 0.680745005607605),
 ('boy', 0.6366527080535889),
 ('lady', 0.6123769879341125),
 ('artist', 0.5911445021629333),
 ('father', 0.5878561735153198),
 ('person', 0.5845483541488647),
 ('men', 0.5828735828399658),
 ('athlete', 0.5750707983970642),
 ('teenager', 0.57445228099823),
 ('preacher', 0.5719428062438965)]

In [13]:
word2vec_own.most_similar(positive=['woman', 'king'], negative=['man'])

[('simon', 0.6156444549560547),
 ('dickens', 0.6015875339508057),
 ('1938', 0.593317985534668),
 ('novella', 0.5894135236740112),
 ('charles', 0.5850018262863159),
 ('hardy', 0.5817059278488159),
 ('jane', 0.580927848815918),
 ('yuppie', 0.580543041229248),
 ('novels', 0.5722631216049194),
 ('lovecraft', 0.571262776851654)]

In [14]:
word2vec_own.init_sims(replace=True)

#### Loading google word2vec

In [15]:
%%time
from gensim.models import Word2Vec

word2vec_google = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
print("Loaded")

Loaded
CPU times: user 2min 22s, sys: 4.27 s, total: 2min 26s
Wall time: 2min 27s


In [16]:
%%time
# Normalizing word2vec vectors.
word2vec_google.init_sims(replace=True)  # Normalizes the vectors in the word2vec class.

CPU times: user 19 s, sys: 2.49 s, total: 21.5 s
Wall time: 21.7 s


In [17]:
len(word2vec_google.index2word)

3000000

In [118]:
word2vec_google.most_similar('Crosby')

[('Malkin', 0.6326718330383301),
 ('Sidney_Crosby', 0.6087846755981445),
 ('Doughty', 0.5953410863876343),
 ('Staal', 0.5891959071159363),
 ('Lemieux', 0.5863054990768433),
 ('Osgood', 0.585029661655426),
 ('Spezza', 0.5774127840995789),
 ('Brodeur', 0.5747572183609009),
 ('Ovechkin', 0.572833776473999),
 ('Letang', 0.5727470517158508)]

In [79]:
word2vec_google.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7118192315101624),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321839332581),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235946178436279),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087412595748901)]

#### 1. String -> Int vector

In [80]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(pd.concat([train_data['prep_text'], test_data['prep_text']]))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 98543 unique tokens.


In [81]:
#save tokenizer
with open(TOKENIZER_NAME,'wb') as ofile:
    pickle.dump(tokenizer, ofile)
    ofile.close()

#### Add embeddings

In [82]:
def get_embedding(word2vec_model, word):
    try:
        return word2vec_model[word]
    except KeyError:
        return np.zeros(word2vec_model.vector_size)

In [83]:
nb_words = min(MAX_NB_WORDS, len(word_index))

embedding_weights_own = np.zeros((nb_words, word2vec_own.vector_size))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_weights_own[i] = get_embedding(word2vec_own, word)
    
embedding_weights_google = np.zeros((nb_words, word2vec_google.vector_size))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_weights_google[i] = get_embedding(word2vec_google, word)

In [84]:
embedding_weights_own.shape

(20000, 300)

#### 2. Padding

In [85]:
sequences_train = tokenizer.texts_to_sequences(train_data['prep_text'])
sequences_test = tokenizer.texts_to_sequences(test_data['prep_text'])

padded_sequences_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
padded_sequences_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)

In [86]:
def padding(text):
    return pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_SEQUENCE_LENGTH)

In [87]:
x_train = padded_sequences_train
x_test = padded_sequences_test
y_train = train_data['label']
y_test= test_data['label']

---

## Model

In [88]:
LSTM_DIM = 128
EMBEDDING_DIM = 300
DROPOUT_U = 0.2
DROPOUT_W = 0.2
DROPOUT_AFTER_LSTM = 0.2

---

In [89]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.layers import Flatten, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

In [90]:
labels = to_categorical(np.asarray(train_data['label']))
print('Shape of data tensor:', padded_sequences_train.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (40000, 50)
Shape of label tensor: (40000, 2)


In [91]:
def create_model(pretrained_embedding_weights = None):
    model = Sequential()
    if pretrained_embedding_weights is not None:
        model.add(Embedding(nb_words,
                            EMBEDDING_DIM, 
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False,
                            weights=[pretrained_embedding_weights]))
    else:
        model.add(Embedding(n_symbols, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(LSTM(LSTM_DIM, dropout_U=DROPOUT_U, dropout_W=DROPOUT_W))
    model.add(Dropout(DROPOUT_AFTER_LSTM))
    model.add(Dense(1, activation='sigmoid'))
    metrics=['accuracy', 'fmeasure', 'precision', 'recall']
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
    return model

In [92]:
lstm_word2vec_own = create_model(embedding_weights_own)
lstm_word2vec_google = create_model(embedding_weights_google)

In [94]:
lstm_word2vec_google.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 50, 300)       6000000     embedding_input_4[0][0]          
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 100)           160400      embedding_4[0][0]                
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 100)           0           lstm_4[0][0]                     
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 1)             101         dropout_4[0][0]                  
Total params: 6,160,501
Trainable params: 160,501
Non-trainable params: 6,000,000
_________

In [95]:
tensor_board = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=False, write_images=False)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model_checkpoint = ModelCheckpoint("models/%s.hdf5" % MODEL_NAME, monitor='val_loss', save_best_only=True, verbose=1)

In [96]:
lstm_word2vec_google.fit(x_train, y_train, 
                         nb_epoch=50,
                         batch_size=128,
                         verbose=2,
                         validation_data=(x_test, y_test),
                         callbacks=[tensor_board, early_stopping, model_checkpoint])

Train on 40000 samples, validate on 10000 samples
Instructions for updating:
Please switch to tf.summary.merge_all.
Epoch 1/50
Epoch 00000: val_loss improved from inf to 0.49658, saving model to models/imdb_lstm_word2vec.hdf5
113s - loss: 0.5843 - acc: 0.6936 - fmeasure: 0.6771 - precision: 0.7207 - recall: 0.6661 - val_loss: 0.4966 - val_acc: 0.7673 - val_fmeasure: 0.7638 - val_precision: 0.7745 - val_recall: 0.7557
Epoch 2/50
Epoch 00001: val_loss improved from 0.49658 to 0.49361, saving model to models/imdb_lstm_word2vec.hdf5
198s - loss: 0.5229 - acc: 0.7415 - fmeasure: 0.7366 - precision: 0.7505 - recall: 0.7306 - val_loss: 0.4936 - val_acc: 0.7651 - val_fmeasure: 0.7568 - val_precision: 0.7824 - val_recall: 0.7352
Epoch 3/50
Epoch 00002: val_loss improved from 0.49361 to 0.47637, saving model to models/imdb_lstm_word2vec.hdf5
146s - loss: 0.5016 - acc: 0.7546 - fmeasure: 0.7509 - precision: 0.7615 - recall: 0.7460 - val_loss: 0.4764 - val_acc: 0.7666 - val_fmeasure: 0.7386 - val_

In [ ]:
%%time
# Fit the model
lstm_word2vec_own.fit(x_train, y_train, 
                         nb_epoch=50,
                         batch_size=128,
                         verbose=2,
                         validation_data=(x_test, y_test),
                         callbacks=[tensor_board, early_stopping, model_checkpoint])

## Analyze results

In [138]:
predictions = lstm_word2vec_google.predict(padded_sequences_test)

In [139]:
test_data['p'] = [p[0] for p in predictions]

In [140]:
test_data[test_data['p'] < 0.1][test_data['label'] == 1][['label', 'p', 'text']].sort_values('p')[:10]

/Users/jenya/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


label         p  \
5281  1      0.000493   
9252  1      0.000627   
8298  1      0.000664   
2453  1      0.003652   
9245  1      0.004152   
8284  1      0.004737   
1122  1      0.005293   
7239  1      0.006486   
9765  1      0.006561   
8916  1      0.006995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [143]:
test_data[test_data['p'] > 0.9][test_data['label'] == 0][['label', 'p', 'text']].sort_values('p', ascending=False)[:10]

/Users/jenya/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,label,p,text
8488,0,0.997479,"By the time this film was released I had seen Chorus Line on stage 4 times, and had been anticipating most eagerly the long-rumored production of a film of the story. My wife and I were in line hours before the box office opened on the day the film was released. It was not just a disappointment, it was a kick in the abdomen. <br /><br />First, the story was ""moved outside,"" so to speak, by including scenes not in the confines of the theater. Those confines are a large portion of the meaning and impact of the story. <br /><br />Second & Third together (assign your own order): one of the original songs, with very dynamic dance number, was removed; a song which was NOT in the stage production was added. Say what ?? I'm confused! <br /><br />The only reason I gave this film 2 stars instead of 1 is my admiration for the talent and hard work of the performers. I've now seen Chorus Line on stage 6 times, and wouldn't mind seeing it 6 more times before I die. It is superbly written, with wonderful music, and heart- wrenchingly true stories. If you want to see a musical which includes a great ""cattle call"" audition, I recommend All That Jazz. If you want to see the story of A Chorus Line, see it on stage."
4198,0,0.995864,"WARNING!!! SOME POSSIBLE PLOT SPOILERS, AS IF THAT WAS POSSIBLE!!!<br /><br />Okay, if you haven't figured out the plot yet, I am going to give it to you. Two couples head out to a camping site, but a clown masked killer shows up and causes all sorts of mayhem.<br /><br />Now it is time for me to give these couples some advice.<br /><br />1. Never, ever go to camp dubbed Camp Blood by the locals. 2. Don't have sex or make out in front of the killer. 3. Don't go chasing after the killer if he or she runs away. 4. Don't yell profanity at the killer. 5. Make sure your cellular phone works before you go camping. 6. Everybody needs to bring a sharp or dangerous weapon and carry it on themselves at all times. 7. Wear loose and comfortable clothing and bring a good pair of running shoes. No tight and binding clothing and no high heels or sandals. 8. Wear glasses or contact lenses if you need them so you don't stab the wrong person. 9. Always carry your car keys on you. 10. Put out a camp fire when you go to bed. 11. Watch your step. If you trip on something, your going to break a bone. 12. Check the backseat for uninvited guess. 13. If the killer has you cornered, at least put up a struggle.<br /><br />And guess what? At least one, but usually all four of the main characters breaks all of these common sense rules. They are the dumbest characters ever to grace a film.<br /><br />Now for some brief comments. This slasher film was on a very tight budget. So low four of the 7 or 8 actors play more than one role. The acting, writing, direction, sound, lighting, camera work, plotting, editing, etc. are all bottom of the line. But like the classic film JACK O this works on a sooooooooo bad, it is very entertaining and campy fun level. So rent this and have a good time and laugh. My rating: 5 out of 10."
5215,0,0.994108,"A young doctor and his wife are suddenly expecting a child. Both are disturbed about a two hour memory lapse on the night of conception. Interesting twist on an hackneyed story. Very good F/X and interesting editing. Jillian McWhirter is outstanding in a cast that features Arnold Vosloo, Wilford Brimley and Brad Dourif. Brimley brings normalcy to the outlandish. Kudos to director Brian Yuzna."
3737,0,0.992040,"I first rented this movie on the infamous day of September 11, 2001. Since then I've seen it a number of times. My only complaint is that it's too short. ""Strangeland"" would've be a complete piece of horror art at two hours. As it stands, the running time is only an hour and a half.<br /><br />Ex-Twisted Sister member Dee Snider wrote, produced and stars in this 1998 shockfest set in a small Colorado town. He plays Carleton Hendricks, a crazed sadist who has psychotic ideo

#### Model sensitivity

In [98]:
def lstm_predict(text):
    return lstm_word2vec_google.predict(padding(text))[0][0]

In [109]:
lstm_predict('good actors, but bad movie')

0.044665568

In [110]:
lstm_predict('really good actors, but bad movie')

0.071367212

In [141]:
lstm_predict('good actors, but really bad movie')

0.010830084

## Save model

In [105]:
# serialize model to JSON
lstm_word2vec_google_json = lstm_word2vec_google.to_json()
with open(MODEL_NAME + ".json", "w") as json_file:
    json_file.write(lstm_word2vec_google_json)